# Using pre-trained word embeddings

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/05/05<br>
**Last modified:** 2020/05/05<br>
**Description:** Text classification on the Newsgroup20 dataset using pre-trained GloVe word embeddings.

##설정

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## 소개

이 예에서는 사전 훈련된 단어 임베딩을 사용하는 텍스트 분류 모델을 훈련하는 방법을 보여줍니다.

20개의 다른 주제 범주에 속하는 20,000개의 게시판 메시지 세트인 Newsgroup20 데이터 세트로 작업하겠습니다.

사전 훈련된 단어 임베딩의 경우 GloVe 임베딩을 사용 합니다.

[GloVe embeddings](http://nlp.stanford.edu/projects/glove/).

GloVe
-  벡터 차이가 두 단어의 병치에 의해 지정된 의미를 최대한 포착하도록 설계
- 전역 단어-단어 동시 발생 행렬의 0이 아닌 항목에 대해 학습되며, 이 행렬은 주어진 말뭉치에서 단어가 서로 얼마나 자주 동시 발생하는지 표를 작성

## Newsgroup20 데이터 다운로드

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

## 데이터 살펴보기

In [3]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")  #숨김파일 .keras/datasets/20_newgroup/comp.graphics 세로 오름차순정렬된 filenames 확인됨
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])  # 랜덤 값인것처럼 배치되어 있음

Number of directories: 20
Directory names: ['talk.religion.misc', 'talk.politics.mideast', 'sci.crypt', 'misc.forsale', 'comp.os.ms-windows.misc', 'comp.sys.mac.hardware', 'sci.space', 'comp.sys.ibm.pc.hardware', 'sci.electronics', 'rec.sport.hockey', 'talk.politics.misc', 'talk.politics.guns', 'rec.sport.baseball', 'alt.atheism', 'rec.autos', 'comp.graphics', 'sci.med', 'comp.windows.x', 'rec.motorcycles', 'soc.religion.christian']
Number of files in comp.graphics: 1000
Some example filenames: ['38596', '38770', '38333', '38432', '38744']


다음은 한 파일에 포함된 내용의 예입니다.

In [4]:
print(open(data_dir / "comp.graphics" / "38987").read()) 

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



Organization: Rensselaer Polytechnic Institute, Troy, NY. ->조직: 렌셀러 폴리테크닉 인스티튜트, 트로이, 뉴욕.

보시다시피 명시적으로(첫 번째 줄은 말 그대로 범주 이름임) 또는 암시적으로(예: 파일을 통해) 파일의 범주를 누출하는 헤더 줄이 있습니다 Organization. 헤더를 제거합시다.

In [5]:
# 샘플 목록에 for 문을 활용하여 라벨, 클래스명, 클래스 인덱스 형식으로 정리, 클래스명, 샘플수 확인
samples = []
labels = []
class_names = []
class_index = 0

for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

실제로 예상되는 파일 수가 없는 범주가 하나 있지만 그 차이가 충분히 작아 문제가 균형 잡힌 분류 문제로 남아 있습니다.

## 데이터를 섞고 훈련 및 검증 세트로 분할

In [6]:
# Shuffle the data
seed = 1337  # seed 어떤 값을 넣던지 별 차이 없이 numpy로 하여금 서로 다른 유사난수 생성
rng = np.random.RandomState(seed) #np.random.RandomState() 
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2  #유효성 검사를 위해 데이터의 20%를 사용
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

<b>np.random.RandomState()Permalink</b>
- 특정 seed를 가지는 np.random.RandomState()를 만들어주고, 여기서부터 이 object에 접근하여 난수를 생성

<b>np.random.random()</b>
- numpy에 존재하는 random generator에 직접 접근하여, 난수를 생성

## 어휘 색인 만들기

<b>TextVectorization</b>
- 데이터 세트에서 찾은 어휘를 색인화 하는 데 사용, 추후 동일 레이어 인스턴트사용 샘플 벡터화

In [7]:
from tensorflow.keras.layers import TextVectorization

# 레이어의 상위 2만단어 고려, 시퀀스를 자르거나 채워서 실제로 200개 토큰 길이 맞추기
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)

# 대용량의 데이터에서 배치 단위로 데이터를 가져오기 위한 데이터셋 제너레이터를 생성하는 코드
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)

vectorizer.adapt(text_ds)

이를 통해 사용된 계산 어휘를 검색 할 수 있습니다.  `vectorizer.get_vocabulary()`. 상위 5개 단어를 인쇄 해보기

In [8]:
# 상위 5개 어휘 검색

vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

테스트 문장을 벡터화합시다

In [9]:
# 테스트 문장 벡터화하기

output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3315, 1823,   15,    2, 6018])

보시다시피 ""는 "2"로 표시됩니다. ""가 어휘의 첫 번째 단어라는 점을 감안할 때 0이 아닌 이유는 무엇입니까? 인덱스 0은 패딩용으로 예약되어 있고 인덱스 1은 "어휘 외" 토큰용으로 예약되어 있기 때문입니다.

다음은 단어를 인덱스에 매핑하는 사전입니다.

In [10]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

보시다시피 테스트 문장에 대해 위와 동일한 인코딩을 얻습니다.

In [11]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3315, 1823, 15, 2, 6018]

## 사전 훈련된 단어 임베딩 로드

사전 훈련된 GloVe 임베딩(822M zip 파일)을 다운로드해 보겠습니다.

다음 명령을 실행해야 합니다.

```
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
```

아카이브에는 50차원, 100차원, 200차원, 300차원 등 다양한 크기의 텍스트로 인코딩된 벡터가 포함되어 있습니다. 우리는 100D를 사용할 것입니다.

NumPy 벡터 표현에 단어(문자열)를 매핑하는 딕셔너리를 만들어 보겠습니다.

In [12]:
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "aiffel/nlp/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


이제 Keras Embedding레이어 에서 사용할 수 있는 해당 임베딩 매트릭스를 준비하겠습니다 . 그것은 인덱스 항목이 단순 NumPy와 매트릭스의 i인덱스의 말씀에 대한 사전 훈련 벡터이다 i우리의 vectorizer의 어휘.

In [13]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


Converted 17998 words (2002 misses)


다음으로 사전 훈련된 단어 임베딩 행렬을 Embedding레이어에 로드합니다 .

trainable=False임베딩을 고정된 상태로 유지하도록 설정했습니다 (교육 중에 업데이트하지 않으려고 함).

In [14]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

## 모델 구축

전역 최대 풀링과 끝에 분류기가 있는 간단한 1D convnet.

In [15]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000200   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         82048 

## 모델구축

먼저 문자열 목록 데이터를 정수 인덱스의 NumPy 배열로 변환합니다. 배열은 오른쪽으로 채워집니다.

In [16]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

우리는 softmax 분류를 수행하기 때문에 범주형 교차 엔트로피를 손실로 사용합니다. 또한 sparse_categorical_crossentropy레이블이 정수이기 때문에 사용 합니다.

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 9s 64ms/step - loss: 2.6689 - acc: 0.1325 - val_loss: 2.0422 - val_acc: 0.2896
Epoch 2/20
125/125 [==============================] - 8s 62ms/step - loss: 1.9441 - acc: 0.3266 - val_loss: 1.6753 - val_acc: 0.4369
Epoch 3/20
125/125 [==============================] - 8s 62ms/step - loss: 1.5270 - acc: 0.4731 - val_loss: 1.3630 - val_acc: 0.5269
Epoch 4/20
125/125 [==============================] - 8s 63ms/step - loss: 1.2655 - acc: 0.5697 - val_loss: 1.1894 - val_acc: 0.5976
Epoch 5/20
125/125 [==============================] - 8s 63ms/step - loss: 1.0956 - acc: 0.6292 - val_loss: 1.1345 - val_acc: 0.6207
Epoch 6/20
125/125 [==============================] - 8s 62ms/step - loss: 0.9668 - acc: 0.6720 - val_loss: 1.1254 - val_acc: 0.6227
Epoch 7/20
125/125 [==============================] - 8s 62ms/step - loss: 0.8533 - acc: 0.7072 - val_loss: 1.0697 - val_acc: 0.6552
Epoch 8/20
125/125 [==============================] - 8s 62ms/step - 

## 종단 간 모델 내보내기

이제 Model일련의 인덱스가 아니라 임의 길이의 문자열을 입력 으로 사용하는 객체 를 내보내고 싶을 수 있습니다 . 입력 전처리 파이프라인에 대해 걱정할 필요가 없기 때문에 모델을 훨씬 더 이식성 있게 만들 수 있습니다.

우리 vectorizer는 실제로 Keras 레이어이므로 간단합니다.

In [ ]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]